In [18]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import easydict
import random
import pickle
from PIL import Image
from collections import defaultdict
import re

In [19]:
path = '/home/anurag'

In [20]:
path_output_chd = path+'/Med_VQA'

In [21]:
import json
with open(path+'/Med_VQA/answer_classes.json', 'r') as j:
        answer_classes = json.load(j)


l = len(answer_classes) 

print(l)

178


In [22]:
opt = easydict.EasyDict({
        "SEED":97,
        "BATCH_SIZE": 32,
        "VAL_BATCH_SIZE": 32,
        "NUM_OUTPUT_UNITS": l,
        "MAX_QUESTION_LEN": 17,
        "IMAGE_CHANNEL": 1472,
        "INIT_LERARNING_RATE": 1e-4,
        "LAMNDA":0.0001,
        "MFB_FACTOR_NUM":5,
        "MFB_OUT_DIM":1000,
        "BERT_UNIT_NUM":768,
        "BERT_DROPOUT_RATIO":0.3,
        "MFB_DROPOUT_RATIO":0.1,
        "NUM_IMG_GLIMPSE":2,
        "NUM_QUESTION_GLIMPSE":2,
        "IMG_FEAT_SIZE":1,
        "IMG_INPUT_SIZE":224,
        "NUM_EPOCHS":100,
    })

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [24]:
path_change = path
seed_value = opt.SEED
print(seed_value) # 97
np.random.seed(seed_value) # return None
random.seed(seed_value) # return None
torch.manual_seed(seed_value) # return <torch._C.Generator object at 0x7f71cdf7a3d0>
torch.cuda.manual_seed(seed_value) # return None
torch.cuda.manual_seed_all(seed_value) # return None
torch.backends.cudnn.enabled = False 
''' backends.cudnn.enabled enables cudnn for some operations such as conv layers and RNNs, which can yield 
a significant speedup. The cudnn RNN implementation doesn’t support the backward operation during eval() 
and thus raises the error. You could disable cudnn for your workload (as already done) or try to call .train()
on the RNN module separately after using model.eval().'''
torch.backends.cudnn.benchmark = False
'''If your model does not change and your input sizes remain the same - then you may benefit from setting 
torch.backends.cudnn.benchmark = True.However, if your model changes: for instance, if you have layers that
are only "activated" when certain conditions are met, or you have layers inside a loop that can be iterated a 
different number of times, then setting torch.backends.cudnn.benchmark = True might stall your execution.'''
torch.backends.cudnn.deterministic = False
'''torch.backends.cudnn.deterministic=True only applies to CUDA convolution operations, and nothing else. 
Therefore, no, it will not guarantee that your training process is deterministic, since you're also using
torch.nn.MaxPool3d, whose backward function is nondeterministic for CUDA.'''

97


"torch.backends.cudnn.deterministic=True only applies to CUDA convolution operations, and nothing else. \nTherefore, no, it will not guarantee that your training process is deterministic, since you're also using\ntorch.nn.MaxPool3d, whose backward function is nondeterministic for CUDA."

In [25]:
#Extract image feature
class VGG19(nn.Module):
    def __init__(self):
        '''
             We remove all the fully-connected layers in the VGG19 network and the convolution outputs of different feature scales
                are concatenated after global average pooling and l2-norm to form a 1984-dimensional vector to represent the image
        '''
        super(VGG19,self).__init__()
        vgg_model = torchvision.models.vgg19(pretrained=True)	
        # resnet_model = torchvision.models.resnet(pretrained=True)

        self.Conv1 = nn.Sequential(*list(vgg_model.features.children())[0:4])
        self.Conv2 = nn.Sequential(*list(vgg_model.features.children())[4:9])
        self.Conv3 = nn.Sequential(*list(vgg_model.features.children())[9:16])
        self.Conv4 = nn.Sequential(*list(vgg_model.features.children())[16:23])
        self.Conv5 = nn.Sequential(*list(vgg_model.features.children())[23:30])
        self.Conv6 = nn.Sequential(*list(vgg_model.features.children())[30:36])

        self.avgpool = nn.Sequential(list(vgg_model.children())[1])

        # self.inc = nn.Linear(1984,2048)
    
    def forward(self,image):

        with torch.no_grad():
            out1 = self.Conv1(image)
            out2 = self.Conv2(out1)
            out3 = self.Conv3(out2)
            out4 = self.Conv4(out3)          
            out5 = self.Conv5(out4)          # [N, 512, 14, 14]
            out6 = self.Conv6(out5) 
            out7 = self.avgpool(out6)

        #global average pooling
        out1 = out1.mean([2,3],keepdim=True)
        out2 = out2.mean([2,3],keepdim=True)
        out3 = out3.mean([2,3],keepdim=True)
        out4 = out4.mean([2,3],keepdim=True)
        out5 = out5.mean([2,3],keepdim=True)
        out6 = out6.mean([2,3],keepdim=True)
        out7 = out7.mean([2,3],keepdim=True)

       
         
        concat_features = torch.cat([out1,out2, out3, out4,out5,out6 ], 1) 
        # print('concat_features',concat_features.shape) # 1,1984,1,1
        # with torch.no_grad():
        #     concat_features = self.inc(torch.squeeze(concat_features))
        
        # concat_features = torch.unsqueeze(concat_features, 0)
        # concat_features = torch.unsqueeze(concat_features, 2)
        # concat_features = torch.unsqueeze(concat_features, 3)

        #l2-normalized feature vector
        l2_norm = concat_features.norm(p=2, dim=1, keepdim=True).detach() 
        concat_features = concat_features.div(l2_norm)               # l2-normalized feature vector
       

        batch_size = concat_features.shape[0]
        embedding_dim_size = concat_features.shape[1]
        image_feature = concat_features.view(batch_size, embedding_dim_size, -1).squeeze(0) # [N, 1984, 1]

        # print('image_feature shape',image_feature.shape)

        return image_feature

In [26]:
vgg19_model = VGG19().to(device)

In [27]:
train_vqa = np.load('/home/anurag/Med_VQA/train_dataset_pickle/train19_subset20_21_df.pkl', allow_pickle=True )

In [28]:
train_vqa.shape

(14216, 5)

In [30]:
size = opt.IMG_INPUT_SIZE

In [31]:
train_transform = transforms.Compose([transforms.RandomResizedCrop(size), 
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),
                                                    (0.229, 0.224, 0.225))])

valid_transform = transforms.Compose([transforms.RandomResizedCrop(size), 
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),
                                                    (0.229, 0.224, 0.225))])

test_transform = transforms.Compose([transforms.RandomResizedCrop(size),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),
                                                    (0.229, 0.224, 0.225))])

In [32]:
train_image_feat={}

In [33]:
for row in train_vqa.itertuples():
    # print(row)
    image = Image.open(row.PATH).convert('RGB')
    image = train_transform(image)
    # print(type(image))
    image_feature = vgg19_model(image[None,...].to(device))
    train_image_feat[row.ID] = image_feature.cpu().numpy()

In [34]:
with open(path_output_chd+'/train_dataset_pickle/train-image-feature-19-subset20.pickle', 'wb') as f:

    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(train_image_feat, f, pickle.HIGHEST_PROTOCOL)

In [35]:
val_vqa = np.load('/home/anurag/Med_VQA/valid_dataset_pickle/val19_df.pkl',allow_pickle=True)

In [36]:
val_vqa.shape

(2000, 5)

In [37]:
valid_image_feat={}

In [38]:
for row in val_vqa.itertuples():
    # print(row)
    image = Image.open(row.PATH).convert('RGB')
    image = train_transform(image)
    # print(type(image))
    image_feature = vgg19_model(image[None,...].to(device))
    valid_image_feat[row.ID] = image_feature.cpu().numpy()

In [39]:
with open(path_output_chd+'/valid_dataset_pickle/valid-image-feature-19-subset20.pickle', 'wb') as f:

    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(valid_image_feat, f, pickle.HIGHEST_PROTOCOL)

In [41]:
test_vqa = np.load('/home/anurag/Med_VQA/test_dataset_pickle/test19_df.pkl',allow_pickle=True)

In [42]:
test_vqa.shape

(500, 5)

In [43]:
test_image_feat={}

In [44]:
for row in test_vqa.itertuples():
    # print(row)
    image = Image.open(row.PATH).convert('RGB')
    image = test_transform(image)
    # print(type(image))
    image_feature = vgg19_model(image[None,...].to(device))
    test_image_feat[row.ID] = image_feature.cpu().numpy()

In [45]:
with open(path_output_chd+'/test_dataset_pickle/test-image-feature-19.pickle', 'wb') as f:

    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(test_image_feat, f, pickle.HIGHEST_PROTOCOL)